### Please run the following cell once.

This will install the Apache Bahir connector within your Project/Apache Spark service. If you restart the kernel or start a new notebook in the same project you can use Apache Bahir for connecting to the Cloudant/Apache CouchDB service. Please note that this will install a patched version of the connector (since the pull request hasn't been merged with the trunk yet).

You'll find more information on the patch here:

https://github.com/apache/bahir/pull/49 https://issues.apache.org/jira/browse/BAHIR-130


In [1]:
!pip install cloudant

In [2]:
#PLEASE INSERT TO CREDENTIALS TO CLOUDANT HERE USING THE IBM WATSON STUDIO CONNECTIONS TAB RIGHT TO THIS NOTEBOOK


In [4]:
from cloudant.client import Cloudant
from cloudant.error import CloudantException
from cloudant.result import Result, ResultByKey

serviceUsername= credentials_1['username']
servicePassword= credentials_1['password']
serviceURL=credentials_1['custom_url']
cloudantName='harlemshake'

client = Cloudant(serviceUsername, servicePassword, url=serviceURL)
client.connect()

In [5]:
df=client[cloudantName]

#for doc in df:
#    print(doc)

In [6]:
from numpy.lib.scimath import sqrt
from pprint import pprint as pp


energy=dict()

# select sqrt(sum(X*X)+sum(Y*Y)+sum(Z*Z)) as energy, SENSORID from data group by SENSORID order by energy desc
for doc in df:
    e=(doc['X']*doc['X'])+(doc['Y']*doc['Y'])+(doc['Z']*doc['Z'])
    if(doc['SENSORID'] not in energy):
        energy.update({doc['SENSORID']:e})
    else:
        energy.update({doc['SENSORID']:e+energy[doc['SENSORID']]})

for key, val in energy.items():
    energy.update({key:sqrt(val)})
    
res=sorted(energy.items(),key=(lambda x:x[1]),reverse=True)

pp(res)

[('daesung', 151.04032838947356)]


## 위의 숫자가 스마트폰의 운동 에너지의 최대값($\sqrt{X^{2}+Y^{2}+Z^{2}}$)입니다.